# <center> Лабораторна робота №11. Налаштування гіперпараметрів регресійних моделей для оцінювання якості вина


In [34]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LinearRegression, LassoCV, Lasso
from sklearn.ensemble import RandomForestRegressor

У завданні буде використано набір даних про якість білого вина(репозиторій UCI)
archive.ics.uci.edu/ml/machine-learning-databases/wine-quality.
Завантажте дані

In [35]:
import os

# Создаём директорию 'data', если её нет
os.makedirs('data', exist_ok=True)


In [48]:
import requests

# URL файла
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv'

# Путь для сохранения файла
file_path = 'data/winequality-white.csv'

# Загрузка файла
response = requests.get(url)
if response.status_code == 200:
    with open(file_path, 'wb') as f:
        f.write(response.content)
    print(f"Файл успешно загружен и сохранён как {file_path}")
else:
    print(f"Не удалось загрузить файл. Статус код: {response.status_code}")


Файл успешно загружен и сохранён как data/winequality-white.csv


In [49]:
data = pd.read_csv("data/winequality-white.csv", sep=";")
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [50]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         4898 non-null   float64
 1   volatile acidity      4898 non-null   float64
 2   citric acid           4898 non-null   float64
 3   residual sugar        4898 non-null   float64
 4   chlorides             4898 non-null   float64
 5   free sulfur dioxide   4898 non-null   float64
 6   total sulfur dioxide  4898 non-null   float64
 7   density               4898 non-null   float64
 8   pH                    4898 non-null   float64
 9   sulphates             4898 non-null   float64
 10  alcohol               4898 non-null   float64
 11  quality               4898 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 459.3 KB


In [51]:
print(data.columns)

Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality'],
      dtype='object')


Відокремте цільову змінну, розділіть навчальну вибірку у відношенні 7:3 (30% - під задишену вибірку, нехай random_state=17) і нормалізуйте дані за допомогою StandartScaler


In [52]:
y = data["quality"]
X = data.drop("quality", axis=1)

X_train, X_holdout, y_train, y_holdout = train_test_split(
    X, y, test_size=0.3, random_state=17
)
scaler = StandardScaler()
# Нормализация тренировочных данных
X_train_scaled = scaler.fit_transform(X_train)

# Нормализация holdout выборки
X_holdout_scaled = scaler.transform(X_holdout)

## Лінійна регресія


Навчіть просту лінійну модель регресії

In [54]:
linreg = LinearRegression()
linreg.fit(X_train_scaled, y_train)

LinearRegression()

> **Питання 1 : Які середньоквадратичні помилки лінійної регресії на навчальній і відкоаденій вибірках ?**

In [55]:
print("Mean squared error (train): %.3f" % mean_squared_error(y_train, linreg.predict(X_train_scaled)))
print("Mean squared error (test): %.3f" % mean_squared_error(y_holdout, linreg.predict(X_holdout_scaled)))

Mean squared error (train): 0.558
Mean squared error (test): 0.584


Подивіться на коефіцієнти моделі і ранжуйте ознаки за впливом на якість вина (врахуйте, що великі за модулем негативні значення коефіцієнтів теж говорять про сильний вплив). Створіть для цього новий невеликий DataFrame.
> **Питання 2 : Яку ознаку лінійна регресія вважає найбільш впливовою на якість вина?**




In [56]:
linreg_coef = pd.DataFrame({"Feature": X.columns, "Coefficient": linreg.coef_})
linreg_coef.sort_values(by="Coefficient", key=abs, ascending=False)

,Feature,Coefficient
7,density,-0.665720
3,residual sugar,0.538164
1,volatile acidity,-0.192260
8,pH,0.150036
10,alcohol,0.129533
0,fixed acidity,0.097822
9,sulphates,0.062053
5,free sulfur dioxide,0.042180
6,total sulfur dioxide,0.014304
4,chlorides,0.008127


## Lasso-регресія

**Навчіть Lasso-регресію з невеликим коефіцієнтом alpha=0,01 (слабка регуляризація). Нехай знову random_state=17.**


In [57]:
lasso1 = Lasso(alpha=0.01, random_state=17)
lasso1.fit(X_train_scaled, y_train)

Lasso(alpha=0.01, random_state=17)

**Подивіться на коефіцієнти моделі і ранжуйте ознаки за впливом на якість вина. Яка ознака "відпала" першою, тобто найменш важлива для пояснення цільової змінної в моделі Lasso?**

In [58]:
lasso1_coef = pd.DataFrame({"Feature": X.columns, "Coefficient": lasso1.coef_})
lasso1_coef.sort_values(by="Coefficient", key=abs, ascending=False)

,Feature,Coefficient
10,alcohol,0.322425
3,residual sugar,0.256363
7,density,-0.235492
1,volatile acidity,-0.188479
8,pH,0.067277
5,free sulfur dioxide,0.043088
9,sulphates,0.029722
4,chlorides,-0.002747
0,fixed acidity,-0.000000
2,citric acid,-0.000000


**Тепер визначте краще значення alpha в процесі 5-кратної крос-валідації. Використовуйте LassoCV і random_state = 17.**

In [59]:
alphas = np.logspace(-6, 2, 200)
lasso_cv = LassoCV(alphas=alphas, cv=5, random_state=17)
lasso_cv.fit(X_train_scaled, y_train)

LassoCV(alphas=array([1.00000000e-06, 1.09698580e-06, 1.20337784e-06, 1.32008840e-06,
       1.44811823e-06, 1.58856513e-06, 1.74263339e-06, 1.91164408e-06,
       2.09704640e-06, 2.30043012e-06, 2.52353917e-06, 2.76828663e-06,
       3.03677112e-06, 3.33129479e-06, 3.65438307e-06, 4.00880633e-06,
       4.39760361e-06, 4.82410870e-06, 5.29197874e-06, 5.80522552e-06,
       6.36824994e-06, 6.98587975e-0...
       1.18953407e+01, 1.30490198e+01, 1.43145894e+01, 1.57029012e+01,
       1.72258597e+01, 1.88965234e+01, 2.07292178e+01, 2.27396575e+01,
       2.49450814e+01, 2.73644000e+01, 3.00183581e+01, 3.29297126e+01,
       3.61234270e+01, 3.96268864e+01, 4.34701316e+01, 4.76861170e+01,
       5.23109931e+01, 5.73844165e+01, 6.29498899e+01, 6.90551352e+01,
       7.57525026e+01, 8.30994195e+01, 9.11588830e+01, 1.00000000e+02]),
        cv=5, random_state=17)

In [60]:
lasso_cv.alpha_

np.float64(0.0002833096101839324)

Виведіть коефіцієнти "кращого" Lasso в порядку зменшення впливу на якість вина.
> **Питання 3: Яка ознака "занулилася першою" в налаштованій моделі LASSO?**

In [61]:
lasso_cv_coef = pd.DataFrame({"Feature": X.columns, "Coefficient": lasso_cv.coef_})
lasso_cv_coef.sort_values(by="Coefficient", key=abs, ascending=False)

,Feature,Coefficient
7,density,-0.648161
3,residual sugar,0.526883
1,volatile acidity,-0.192049
8,pH,0.146549
10,alcohol,0.137115
0,fixed acidity,0.093295
9,sulphates,0.060939
5,free sulfur dioxide,0.042698
6,total sulfur dioxide,0.012969
4,chlorides,0.006933


**Оцініть середньоквадратичну помилку моделі на навчальній і тестовій вибірках.**

> **Питання 4 : Які середньоквадратичні помилки налаштованої LASSO-регресії на навчальній і відкладеній вибірках?**

In [62]:
print("Mean squared error (train): %.3f" % mean_squared_error(y_train, lasso_cv.predict(X_train_scaled)))
print("Mean squared error (test): %.3f" % mean_squared_error(y_holdout, lasso_cv.predict(X_holdout_scaled)))

Mean squared error (train): 0.558
Mean squared error (test): 0.583


## Випадковий ліс

**Навчіть випадковий ліс з параметрами "з коробки", фіксуючи тільки random_state=17.**

In [63]:
forest = RandomForestRegressor(random_state=17)
forest.fit(X_train_scaled, y_train)

RandomForestRegressor(random_state=17)

> **Питання 5 : Які середньоквадратичні помилки випадкового лысу на навчальній вибірці, на крос-валідації (cross_val_score з scoring='neg_mean_squared_error' і іншими параметрами за замовчуванням) і відкладеній вибірках?**

In [64]:
print("Mean squared error (train): %.3f" % mean_squared_error(y_train, forest.predict(X_train_scaled)))
print("Mean squared error (cv): %.3f" % -cross_val_score(forest, X_train_scaled, y_train, cv=5, scoring='neg_mean_squared_error').mean())
print("Mean squared error (test): %.3f" % mean_squared_error(y_holdout, forest.predict(X_holdout_scaled)))

Mean squared error (train): 0.053
Mean squared error (cv): 0.414
Mean squared error (test): 0.372


**Налаштуйте параметри min_samples_leaf і max_depth за допомогою GridSearchCV і знову перевірте якість моделі на крос-валідації і на відкладеній вибірках.**

In [65]:
forest_params = {'max_depth': list(range(10, 25)),
                 'min_samples_leaf': list(range(1, 8)),
                 'max_features': list(range(6, 12))}
locally_best_forest = GridSearchCV(forest, forest_params, cv=5, scoring='neg_mean_squared_error')
locally_best_forest.fit(X_train_scaled, y_train)

c:\Users\Dmitry\Documents\Repos\Data_Science_Intro\.venv312\Lib\site-packages\numpy\ma\core.py:2881: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


GridSearchCV(cv=5, estimator=RandomForestRegressor(random_state=17),
             param_grid={'max_depth': [10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
                                       20, 21, 22, 23, 24],
                         'max_features': [6, 7, 8, 9, 10, 11],
                         'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7]},
             scoring='neg_mean_squared_error')

In [66]:
locally_best_forest.best_params_, locally_best_forest.best_score_

({'max_depth': 21, 'max_features': 6, 'min_samples_leaf': 1},
 np.float64(-0.39773288191505934))

**Нажал результати GridSearchCV в повному не відтворювані (можуть відрізнятися на різних платформах навіть при фіксованому random_state). Тому навчіть ліс з параметрами max_depth=19, max_features=7, i min_samples_leaf=1 (краще в моэму випадку).**
> **Питання 6 : Які середньоквадратичні помилки налаштованого випадкового лісу на навчальній вибірці, на крос-валідації (cross_val_score з scoring='neg_mean_squared_error') і на відкладеній вибірках?**


In [67]:
print("Mean squared error (cv): %.3f" % -cross_val_score(forest, X_train_scaled, y_train, cv=5, scoring='neg_mean_squared_error').mean())
print("Mean squared error (test): %.3f" % mean_squared_error(y_holdout, forest.predict(X_holdout_scaled)))

Mean squared error (cv): 0.414
Mean squared error (test): 0.372


**Оцініть важливість ознак за допомогою випадкового лісу.**
>**Питання 7 : Яка ознака виявилася найінформативнішою в налаштованій моделі випадкового лісу?**

In [68]:
rf_importance = pd.DataFrame({"Feature": X.columns, "Importance": forest.feature_importances_})
rf_importance.sort_values(by="Importance", ascending=False)

,Feature,Importance
10,alcohol,0.247273
1,volatile acidity,0.125960
5,free sulfur dioxide,0.116562
8,pH,0.071950
3,residual sugar,0.070903
6,total sulfur dioxide,0.067204
4,chlorides,0.063729
0,fixed acidity,0.061918
9,sulphates,0.059056
2,citric acid,0.058904


**Зробіть висновки про якість моделей і оцінки впливу ознак на якість вина за допомогою цих трьох моделей**

Висновки щодо якості моделей та впливу ознак на якість вина:
Лінійна регресія:

Середньоквадратична помилка (MSE):
Навчальна вибірка: 0.558
Тестова вибірка: 0.584
Аналіз якості:
Лінійна регресія показала середній рівень якості прогнозів. Різниця між помилками на навчальній і тестовій вибірках незначна, що вказує на відсутність явного переобучення, проте точність залишає бажати кращого.
Оцінка впливу ознак:
Найбільш впливовою ознакою є density (щільність) з коефіцієнтом -0.665720, що має сильний негативний вплив на якість вина.
Також значний позитивний вплив має residual sugar (залишковий цукор) — 0.538164.
Lasso-регресія:

Середньоквадратична помилка (MSE):
Навчальна вибірка: 0.558
Тестова вибірка: 0.583
Аналіз якості:
Результати Lasso-регресії майже такі ж, як і у звичайної лінійної регресії, що вказує на те, що слабка регуляризація (alpha=0.01) не покращила значно якість моделі.
Lasso регресія занулила кілька ознак, зокрема fixed acidity, citric acid, і total sulfur dioxide, що свідчить про їх низьку важливість для моделі.
Оцінка впливу ознак:
Найбільш впливовою ознакою є alcohol (алкоголь) з коефіцієнтом 0.322425, що має позитивний вплив на якість вина.
residual sugar (залишковий цукор) і density (щільність) також залишаються серед ключових ознак.
Випадковий ліс:

Середньоквадратична помилка (MSE):
Навчальна вибірка: 0.053
Крос-валідація: 0.414
Тестова вибірка: 0.372
Аналіз якості:
Випадковий ліс демонструє значно кращі результати в порівнянні з лінійною і Lasso-регресією, особливо на тестовій вибірці (MSE = 0.372), що вказує на його здатність краще враховувати складні взаємозв'язки між ознаками.
Проте є певне переобучення (MSE на навчальній вибірці = 0.053), але воно незначне, оскільки результати на крос-валідації і тестовій вибірці близькі.
Оцінка впливу ознак:
Найбільш інформативними ознаками виявилися ті ж, що і в інших моделях: alcohol (алкоголь), residual sugar (залишковий цукор), і density (щільність). Однак випадковий ліс враховує більше складних взаємодій між ознаками.
Загальні висновки:
Лінійна регресія і Lasso-регресія мають схожі результати і демонструють середню точність, причому Lasso вказала на те, що деякі ознаки є менш значущими.
Випадковий ліс показав кращу якість моделі і краще справляється з передбаченням якості вина. Він виявився найкращим серед трьох моделей з точки зору середньоквадратичної помилки на тестовій вибірці (0.372).
Всі моделі вказують на те, що найбільш значущими ознаками для прогнозування якості вина є alcohol, residual sugar, і density.